In [3]:
import pandas as pd
df = pd.read_html('https://es.wikipedia.org/wiki/Regi%C3%B3n_de_Los_Lagos')[3]
df.drop(columns=['Bandera', 'Escudo'], inplace=True)
df['Comunas'] = df.apply(lambda row: row['Comunas'][2:], axis=1)
df.index += 1 

</br></br></br></br>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Los_Lagos_Comunas.svg/450px-Los_Lagos_Comunas.svg.png" width="120%" />

Fuente: [wikipedia](https://es.wikipedia.org/wiki/Portal:Regi%C3%B3n_de_Los_Lagos/Comunas_de_Los_Lagos)

In [4]:
df

,Provincia,Capital,Comunas,Población
1,Chiloé,Castro,Ancud,38.991
2,Chiloé,Castro,Castro,33.417
3,Chiloé,Castro,Chonchi,5.632
4,Chiloé,Castro,Curaco de Vélez,3.829
5,Chiloé,Castro,Dalcahue,7.120
6,Chiloé,Castro,Puqueldón,3.921
7,Chiloé,Castro,Queilén,5.385
8,Chiloé,Castro,Quemchi,8.352
9,Chiloé,Castro,Quellón,27.192
10,Chiloé,Castro,Quinchao,8.088
